# Interpolation

### Prerequisites

**Imports**

In [ ]:
import os
import sys
import warnings
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt

**Fix directories, define path variables**

In [ ]:
# make sure notebook is ran from src
cwd = os.getcwd()
if not cwd.split('/')[-1] == 'src':
    try:
        os.chdir('src')
    except FileNotFoundError:
        print('Error: please run from src dir or project root')
        sys.exit(1)

# define paths
data_path = '../data/'
csv_path = data_path + 'csv/'
shp_path = data_path + 'shapefiles/'
plot_path = '../plots/'
if not os.path.exists(plot_path):
    os.mkdir(plot_path)

---

### Read data

In [ ]:
# streets data
gdf_streets = gpd.read_file(shp_path + 'streets/M_streets.shp')

# traffic data
gdf_traffic = gpd.read_file(shp_path + 'traffic/M_traffic.shp')
gdf_traffic.info()

---

### Interpolate traffic data

In [ ]:
x, y = gdf_traffic.geometry.x.values, gdf_traffic.geometry.y.values
phi = gdf_traffic.Avg_Vol.values

assert x.shape[0] == y.shape[0] == phi.shape[0]

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
cax = ax.scatter(x, y, c=phi, cmap='viridis', s=5, alpha=0.75, zorder=2)
gdf_streets.plot(ax=ax, color='gray', alpha=0.5, zorder=1)
ax.set_title('Observations')
ax.set_axis_off()
fig.colorbar(cax, ax=ax, shrink=0.4)
fig.tight_layout()

In [ ]:
OK = OrdinaryKriging(x, y, phi, variogram_model='exponential', verbose=False, enable_plotting=False)
minx, miny, maxx, maxy = gdf_streets.total_bounds
X = np.arange(minx, maxx, 0.005)
Y = np.arange(miny, maxy, 0.005)
zstar, ss = OK.execute('grid', X, Y)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
cax = ax.imshow(
    zstar,
    extent = (minx, maxx, miny, maxy),
    cmap = 'viridis',
    origin = 'lower',
    alpha = 0.9,
    zorder = 1
)
gdf_streets.plot(
    ax = ax,
    color = 'k',
    alpha = 0.1,
    zorder = 2
)
ax.scatter(x, y, c=phi, cmap='viridis', edgecolors='k', s=10, zorder=3)
ax.set_title('Ordinary Kriging')
fig.colorbar(cax, ax=ax, shrink=0.4)
fig.tight_layout()
fig.savefig(plot_path + 'ordinary_kriging.png', dpi=300)